In [8]:
# Battery Capacity Prediction from EIS Data

import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

metadata_path = "../cleaned_dataset/metadata.csv"
data_dir = "../cleaned_dataset/data/"

metadata = pd.read_csv(metadata_path)

discharge_meta = metadata[metadata["type"] == "discharge"][["start_time", "filename", "Capacity"]].dropna()
impedance_meta = metadata[metadata["type"] == "impedance"][["start_time", "filename", "Re", "Rct"]]

def parse_start_time(value):
    try:
        value = str(value).strip('[]').replace(',', '')
        components = [float(x) for x in value.split()]
        return sum(components) 
    except:
        return np.nan

discharge_meta["start_time"] = discharge_meta["start_time"].apply(parse_start_time)
impedance_meta["start_time"] = impedance_meta["start_time"].apply(parse_start_time)

impedance_meta["Capacity"] = impedance_meta["start_time"].apply(
    lambda t: discharge_meta.iloc[(discharge_meta["start_time"] - t).abs().idxmin()]["Capacity"]
)

def extract_impedance_features(file_path):
    try:
        df = pd.read_csv(file_path)
        
        if "Battery_impedance" not in df.columns:
            return pd.Series({'Re(Z)': np.nan, 'Im(Z)': np.nan})
        
        df['Re(Z)'] = df['Battery_impedance'].apply(lambda x: float(complex(x).real) if '(' in x else np.nan)
        df['Im(Z)'] = df['Battery_impedance'].apply(lambda x: float(complex(x).imag) if '(' in x else np.nan)

        return df[['Re(Z)', 'Im(Z)']].mean()
    
    except:
        return pd.Series({'Re(Z)': np.nan, 'Im(Z)': np.nan})

impedance_data = []
for _, row in impedance_meta.iterrows():
    file_path = os.path.join(data_dir, row["filename"])
    if os.path.exists(file_path):
        features = extract_impedance_features(file_path)
        features['Capacity'] = row['Capacity']  
        impedance_data.append(features)

impedance_df = pd.DataFrame(impedance_data).dropna()

print(impedance_df.head())

X = impedance_df[['Re(Z)', 'Im(Z)']]
y = impedance_df['Capacity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Training Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

# Evaluating Model
y_pred = model.predict(X_test_scaled)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Score: {r2:.4f}")


Extracted 1956 valid impedance samples.
      Re(Z)     Im(Z)              Capacity
0  0.206678  0.004761    1.7502913788006587
1  0.212963 -0.011729    1.7695250734505363
2  0.214454  0.004741    1.5935866593100128
3  0.216954 -0.010778    0.4668172561120673
4  0.211492  0.004910  0.061130459259873135
Mean Absolute Error (MAE): 0.2774
R² Score: -0.0002
